In [1]:
import pandas as pd
import numpy as np
import spacy
from transformers import BertTokenizer, BertModel
import torch
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from transformers import pipeline
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')  # For WordNet Lemmatizer


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/achilleasdel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/achilleasdel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/achilleasdel/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

def news_website_destroyer(url):
    """
    Scrapes news articles from the specified URL and returns a DataFrame.
    It tests multiple HTML structures to maximize the amount of data retrieved.

    Parameters:
    url (str): The URL of the news website to scrape.

    Returns:
    pd.DataFrame or str: A DataFrame containing the scraped articles, or a message if data could not be collected.
    """
    # Custom User-Agent to mimic a web browser #to manage to enter html structure when not alowed
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to retrieve data from {url}. Status code: {response.status_code}")
        return "Data could not be collected"  # Return message if request fails

    soup = BeautifulSoup(response.text,"html.parser") #gets html structure of website

    # common html structures to test (html structure can differ from website to website
    structures = [
        {'article_tag': 'article', 'title_tag': 'h1', 'link_tag': 'a', 'summary_tag': 'p'},
        {'article_tag': 'article', 'title_tag': 'h2', 'link_tag': 'a', 'summary_tag': 'p'},
        {'article_tag': 'div', 'title_tag': 'h1', 'link_tag': 'a', 'summary_tag': 'p'},
        {'article_tag': 'section', 'title_tag': 'h2', 'link_tag': 'a', 'summary_tag': 'p'},
        {'article_tag': 'div', 'title_tag': 'span', 'link_tag': 'a', 'summary_tag': 'p'},

    ]

    articles_data = []  # Prepare to store articles data

    # for loop to test possible html structures
    for structure in structures:
        for article in soup.find_all(structure['article_tag']):
            title = article.find(structure['title_tag']).text.strip() if article.find(structure['title_tag']) else None
            link_tag = article.find(structure['link_tag'])
            link = urljoin(url, link_tag['href']) if link_tag and link_tag.has_attr('href') else 'No Link'

            # Get summary text of the article
            full_text = article.get_text(separator=' ', strip=True)

            # Append the article data as a dictionary to the list
            if title:  # Only append if title is found
                articles_data.append({
                    'Title': title,
                    'Link': link,
                    'Summary': full_text  # Full text as Summary
                })

    # Create a DataFrame from the articles data
    df = pd.DataFrame(articles_data)

    # Check if DataFrame is empty and return the message if no articles were collected
    if df.empty:
        return "Data could not be collected"

    return df


In [3]:
df=news_website_destroyer("https://edition.cnn.com/")
df.head()

,Title,Link,Summary
0,Will China make it a trade war?,https://edition.cnn.com/2025/02/02/politics/us...,Trump tariffs 2025 Grammy Awards red carpet Ma...
1,Will China make it a trade war?,https://edition.cnn.com/2025/02/02/politics/us...,Trump tariffs 2025 Grammy Awards red carpet Ma...
2,Will China make it a trade war?,https://edition.cnn.com/2025/02/02/politics/us...,Trump tariffs 2025 Grammy Awards red carpet Ma...
3,More,https://edition.cnn.com/us,US World Politics Business Health Entertainmen...
4,More,No Link,More US World Politics Business Health Enterta...


In [4]:
print(df.duplicated().sum())

217


In [5]:
#perfect now we have a dataframe with titles , a link to the article and a summary of the article ,also it has many duplicated rows (221)
#lets drop duplicates
df.drop_duplicates(inplace=True)
print(df.duplicated().sum())
#perfect no duplicates left , lets continue our pre processing

0


In [6]:
df[df["Link"]=="No Link"].shape
#We can see that on some rows no links might be retrieved , lets drop those rows


(102, 3)

In [7]:
df.head()


,Title,Link,Summary
0,Will China make it a trade war?,https://edition.cnn.com/2025/02/02/politics/us...,Trump tariffs 2025 Grammy Awards red carpet Ma...
3,More,https://edition.cnn.com/us,US World Politics Business Health Entertainmen...
4,More,No Link,More US World Politics Business Health Enterta...
5,Your CNN account,No Link,Your CNN account Sign in to your CNN account
7,Follow CNN,https://facebook.com/CNN,Follow CNN


In [8]:
# drop all rows that contain no link
df.drop(df[df['Link'] == 'No Link'].index, inplace=True)

df.head()


,Title,Link,Summary
0,Will China make it a trade war?,https://edition.cnn.com/2025/02/02/politics/us...,Trump tariffs 2025 Grammy Awards red carpet Ma...
3,More,https://edition.cnn.com/us,US World Politics Business Health Entertainmen...
7,Follow CNN,https://facebook.com/CNN,Follow CNN
9,Trump tariffs,https://edition.cnn.com/2025/02/02/politics/us...,Trump tariffs 2025 Grammy Awards red carpet Ma...
10,Trump tariffs,https://edition.cnn.com/2025/02/02/politics/us...,Trump tariffs 2025 Grammy Awards red carpet Ma...


In [9]:
df.reset_index(drop=True, inplace=True)
df.head()
#now for the links left lets create a new column "text" that contains the text of the article

,Title,Link,Summary
0,Will China make it a trade war?,https://edition.cnn.com/2025/02/02/politics/us...,Trump tariffs 2025 Grammy Awards red carpet Ma...
1,More,https://edition.cnn.com/us,US World Politics Business Health Entertainmen...
2,Follow CNN,https://facebook.com/CNN,Follow CNN
3,Trump tariffs,https://edition.cnn.com/2025/02/02/politics/us...,Trump tariffs 2025 Grammy Awards red carpet Ma...
4,Trump tariffs,https://edition.cnn.com/2025/02/02/politics/us...,Trump tariffs 2025 Grammy Awards red carpet Ma...


In [10]:
#create function to get link to article and extract article text
import requests
from bs4 import BeautifulSoup

def fetch_article_text(url):
    """
    Fetches the complete text of a news article from the specified URL.

    Parameters:
    url (str): The URL of the news article to scrape.

    Returns:
    str: The complete text of the article, or an error message if the article could not be retrieved.
    """
    # Custom User-Agent to mimic a web browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }

    try:
        # Send a GET request to the article URL
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            return f"Failed to retrieve data. Status code: {response.status_code}"

        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Example: Assuming the main article text is in the <article> tag or similar
        article_content = soup.find('article') or soup.find('div', class_='story')  # Adjust as necessary

        if article_content:
            # Get the full text of the article
            full_text = article_content.get_text(separator=' ', strip=True)
            return full_text
        else:
            return "Article content not found in the specified structure."

    except Exception as e:
        return f"An error occurred: {str(e)}"



In [11]:
#fetch_article_text('https://edition.cnn.com/2024/11/28/china/china-military-general-investigation-intl-hnk/index.html')

In [12]:
for number in range(len(df)):
    print(df["Link"])

0      https://edition.cnn.com/2025/02/02/politics/us...
1                             https://edition.cnn.com/us
2                               https://facebook.com/CNN
3      https://edition.cnn.com/2025/02/02/politics/us...
4      https://edition.cnn.com/2025/02/02/politics/us...
                             ...                        
126    https://edition.cnn.com/2025/01/28/science/bee...
127    https://edition.cnn.com/2025/01/28/politics/tr...
128    https://edition.cnn.com/2025/01/28/health/soci...
129    https://edition.cnn.com/2025/01/24/style/louvr...
130                        https://edition.cnn.com/video
Name: Link, Length: 131, dtype: object
0      https://edition.cnn.com/2025/02/02/politics/us...
1                             https://edition.cnn.com/us
2                               https://facebook.com/CNN
3      https://edition.cnn.com/2025/02/02/politics/us...
4      https://edition.cnn.com/2025/02/02/politics/us...
                             ...                 

In [13]:
len(df)

131

In [14]:
article=[]
for number in range(len(df)):
    article_text = fetch_article_text(df["Link"][number])
    article.append(article_text)


In [15]:
df["article_text"]=article
df.head()

,Title,Link,Summary,article_text
0,Will China make it a trade war?,https://edition.cnn.com/2025/02/02/politics/us...,Trump tariffs 2025 Grammy Awards red carpet Ma...,Video Ad Feedback White House press secretary ...
1,More,https://edition.cnn.com/us,US World Politics Business Health Entertainmen...,Article content not found in the specified str...
2,Follow CNN,https://facebook.com/CNN,Follow CNN,Article content not found in the specified str...
3,Trump tariffs,https://edition.cnn.com/2025/02/02/politics/us...,Trump tariffs 2025 Grammy Awards red carpet Ma...,Video Ad Feedback White House press secretary ...
4,Trump tariffs,https://edition.cnn.com/2025/02/02/politics/us...,Trump tariffs 2025 Grammy Awards red carpet Ma...,Video Ad Feedback White House press secretary ...


In [16]:
import pandas as pd
from gensim import corpora, models
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import re
from gensim.models.coherencemodel import CoherenceModel
from pprint import pprint

# Download necessary NLTK data 
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')


stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text) #remove punctuation
    text = re.sub(r'\d+', '', text) #remove numbers
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    tokens = [w for w in tokens if not w in stop_words and len(w)>2] #remove stop words and short words
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    return tokens





df['processed_text'] = df['article_text'].apply(preprocess_text)


# Create a dictionary and corpus
dictionary = corpora.Dictionary(df['processed_text'])
# Filter out extremes
dictionary.filter_extremes(no_below=15, no_above=0.5) #adjust if necessary
corpus = [dictionary.doc2bow(text) for text in df['processed_text']]

# Function to compute coherence score
def compute_coherence_score(corpus, dictionary, topic_num, model):
    coherence_model = CoherenceModel(model=model, texts=df['processed_text'], dictionary=dictionary, coherence='c_v')
    return coherence_model.get_coherence()


# Find optimal number of topics (adjust range as needed)
best_coherence = -1
best_num_topics = 0
coherence_scores = []
for num_topics in range(2, 11):  # Try different numbers of topics
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15, random_state=42)
    coherence_score = compute_coherence_score(corpus, dictionary, num_topics, lda_model)
    coherence_scores.append(coherence_score)
    if coherence_score > best_coherence:
        best_coherence = coherence_score
        best_num_topics = num_topics

print(f"Best number of topics: {best_num_topics} with coherence score: {best_coherence}")

# Train the LDA model with the optimal number of topics
lda_model = models.LdaModel(corpus, num_topics=best_num_topics, id2word=dictionary, passes=15, random_state=42)


# Print topics with improved formatting
pprint(lda_model.print_topics())


#Assign topics to articles. This time we take into account all topic probabilities
def assign_topic(bow):
    topics = lda_model.get_document_topics(bow, minimum_probability=0)
    if topics: #in case no topic is found
        return max(topics,key=lambda x: x[1])[0]
    else:
        return -1


df['topic'] = [assign_topic(doc) for doc in corpus]
print(df.head())

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/achilleasdel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/achilleasdel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/achilleasdel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Best number of topics: 4 with coherence score: 0.4999297511644627
[(0,
  '0.058*"trump" + 0.051*"tariff" + 0.030*"video" + 0.027*"source" + '
  '0.026*"feedback" + 0.024*"playing" + 0.017*"american" + 0.015*"canada" + '
  '0.012*"mexico" + 0.010*"former"'),
 (1,
  '0.081*"school" + 0.036*"technology" + 0.036*"system" + 0.021*"security" + '
  '0.019*"according" + 0.014*"used" + 0.014*"threat" + 0.013*"measure" + '
  '0.013*"case" + 0.013*"use"'),
 (2,
  '0.010*"life" + 0.008*"show" + 0.007*"much" + 0.007*"found" + 0.007*"way" + '
  '0.007*"food" + 0.007*"study" + 0.007*"two" + 0.006*"love" + 0.006*"around"'),
 (3,
  '0.021*"trump" + 0.011*"show" + 0.011*"best" + 0.011*"president" + '
  '0.009*"government" + 0.009*"january" + 0.009*"federal" + 0.008*"agency" + '
  '0.007*"department" + 0.007*"sunday"')]
                             Title  \
0  Will China make it a trade war?   
1                             More   
2                       Follow CNN   
3                    Trump tariffs 

In [17]:
#df["topic"].head(10)
#!pip install -U langchain-community
df.head()

,Title,Link,Summary,article_text,processed_text,topic
0,Will China make it a trade war?,https://edition.cnn.com/2025/02/02/politics/us...,Trump tariffs 2025 Grammy Awards red carpet Ma...,Video Ad Feedback White House press secretary ...,"[video, feedback, white, house, press, secreta...",0
1,More,https://edition.cnn.com/us,US World Politics Business Health Entertainmen...,Article content not found in the specified str...,"[article, content, found, specified, structure]",2
2,Follow CNN,https://facebook.com/CNN,Follow CNN,Article content not found in the specified str...,"[article, content, found, specified, structure]",2
3,Trump tariffs,https://edition.cnn.com/2025/02/02/politics/us...,Trump tariffs 2025 Grammy Awards red carpet Ma...,Video Ad Feedback White House press secretary ...,"[video, feedback, white, house, press, secreta...",0
4,Trump tariffs,https://edition.cnn.com/2025/02/02/politics/us...,Trump tariffs 2025 Grammy Awards red carpet Ma...,Video Ad Feedback White House press secretary ...,"[video, feedback, white, house, press, secreta...",0


In [37]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# ... (Your LDA code from above) ...


def create_chatbot(df):
    """Creates a chatbot using TF-IDF and  LDA model's topic assignments."""

    df['processed_text'] = df['processed_text'].apply(lambda x: ' '.join(map(str, x)) if isinstance(x, list) else str(x))

    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(df['processed_text'])


    #Simplified topic determination:  Directly use LDA topic
    def determine_topic(question):
        # we  assume that  a simple keyword based matching may not be the most efficient here.
        #Instead, we'll process the question with the same method as used in the LDA and check the highest probability topic.
        processed_question = preprocess_text(question)
        bow_question = dictionary.doc2bow(processed_question)
        if bow_question:
            return assign_topic(bow_question)
        else:
            return -1 #Handle cases where no topic is assigned


    def get_answer(question):
        question = question.lower()
        question_vector = vectorizer.transform([question])
        similarities = cosine_similarity(question_vector, tfidf_matrix).flatten()


        top_topic = determine_topic(question)

        weighted_similarities = similarities.copy()
        topic_match_bonus = 0.3  # Adjust this to tune the effect of topic matching.
        for i, row in df.iterrows():
            if row['topic'] == top_topic:
                weighted_similarities[i] += topic_match_bonus

        best_match_index = weighted_similarities.argmax()
        best_match = df.iloc[best_match_index]

        answer = f"Based on the article about '{best_match['Title']}' (Topic: {best_match['topic']}):\n\n{best_match['Summary']}\n\nFor more details, see: {best_match['Link']}"
        return answer

    return get_answer

# Example usage:
chatbot = create_chatbot(df)

while True:
    question = input("Ask a question (or type 'exit' to quit): ")
    if question.lower() == 'exit':
        break
    try:
        answer = chatbot(question)
        print(answer)
    except (IndexError, KeyError) as e:
        print(f"I couldn't find a relevant answer to your question. Error: {e}")

Ask a question (or type 'exit' to quit):  tell me about Putin


Based on the article about 'Ukrainian troops fear the consequences of a potential ceasefire' (Topic: 3):

Ukrainian troops fear the consequences of a potential ceasefire

For more details, see: https://edition.cnn.com/2025/02/02/europe/ukraine-russia-frontline-ceasefire-deal-intl/index.html


Ask a question (or type 'exit' to quit):  exit
